In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
from scipy import stats
import pandas as pd
import requests
from config import api_key
import numpy as np
from ratelimiter import RateLimiter
from citipy import citipy
from scipy.stats import linregress
import gmaps

# Import API key
from config import gkey

#Access maps with unique API key
gmaps.configure(api_key=gkey)

city_weather_df = pd.read_csv("../WeatherPy/Resources/cities.csv")
city_weather_df


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,78.71,75,50,14.56,PF,1613532705
1,Castro,-24.7911,-50.0119,61.12,93,2,4.14,BR,1613532502
2,Norman Wells,65.2820,-126.8329,-9.40,77,75,3.44,CA,1613532706
3,Aksarka,66.5606,67.7975,-16.19,88,100,4.74,RU,1613532706
4,Ahipara,-35.1667,173.1667,71.67,66,34,7.45,NZ,1613532706
...,...,...,...,...,...,...,...,...,...
472,Manjacaze,-24.7117,33.8828,74.30,92,100,17.11,MZ,1613533191
473,Romhány,47.9262,19.2572,33.01,59,90,4.61,HU,1613533191
474,Ried,48.4770,11.2606,41.00,93,90,10.36,DE,1613533191
475,San Pedro Buenavista,16.0833,-93.1167,75.20,46,1,3.87,MX,1613533192


In [2]:
#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                 max_intensity=np.max(humidity), point_radius=2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [3]:
#Narrow down the cities to fit weather conditions
#Drop any rows will null values
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Clouds"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
53,Pisco,-13.7000,-76.2167,71.60,78,0,5.75,PE,1613532765
59,Simao,22.7886,100.9748,72.75,40,0,4.45,CN,1613532766
125,Arraial do Cabo,-22.9661,-42.0278,75.51,88,0,4.79,BR,1613532829
136,Hambantota,6.1241,81.1185,78.80,94,0,8.05,LK,1613532825
147,São João da Barra,-21.6403,-41.0511,75.67,90,0,2.77,BR,1613532833
201,Cidreira,-30.1811,-50.2056,70.72,84,0,8.63,BR,1613532893
226,Hong Kong,22.2855,114.1577,72.00,62,0,1.01,HK,1613532683
245,General Roca,-39.0333,-67.5833,75.20,35,0,8.05,AR,1613532951
270,Sittwe,20.1500,92.9000,78.30,31,0,2.95,MM,1613532954
333,Anūpgarh,29.1911,73.2086,70.32,22,0,1.16,IN,1613533015


In [4]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
53,Pisco,PE,-13.7000,-76.2167,
59,Simao,CN,22.7886,100.9748,
125,Arraial do Cabo,BR,-22.9661,-42.0278,
136,Hambantota,LK,6.1241,81.1185,
147,São João da Barra,BR,-21.6403,-41.0511,
201,Cidreira,BR,-30.1811,-50.2056,
226,Hong Kong,HK,22.2855,114.1577,
245,General Roca,AR,-39.0333,-67.5833,
270,Sittwe,MM,20.1500,92.9000,
333,Anūpgarh,IN,29.1911,73.2086,


In [5]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

https://maps.googleapis.com/maps/api/place/nearbysearch/json?

Object `json` not found.


In [6]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
#     print(response)
#     print(results).json()
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No results found, moving to the next city ")             
        print("_______________________________________________")

Retrieving Results for Index 53: Pisco.
Closest hotel in Pisco is Hostal Mandala.
Retrieving Results for Index 59: Simao.
Closest hotel in Simao is Urba Hotel Chacheng Pu'er.
Retrieving Results for Index 125: Arraial do Cabo.
Closest hotel in Arraial do Cabo is Orlanova Boutique Hotel.
Retrieving Results for Index 136: Hambantota.
Closest hotel in Hambantota is The Peacock Beach Hotel Hambantota.
Retrieving Results for Index 147: São João da Barra.
Closest hotel in São João da Barra is OYO Pousada Mediterrâneo.
Retrieving Results for Index 201: Cidreira.
Closest hotel in Cidreira is Hotel Castelo.
Retrieving Results for Index 226: Hong Kong.
Closest hotel in Hong Kong is The OTTO Hotel.
Retrieving Results for Index 245: General Roca.
Closest hotel in General Roca is Estepa Hotel.
Retrieving Results for Index 270: Sittwe.
Closest hotel in Sittwe is Hotel Memory.
Retrieving Results for Index 333: Anūpgarh.
Closest hotel in Anūpgarh is Hotel shiva.
Retrieving Results for Index 392: São Se

In [7]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
53,Pisco,PE,-13.7000,-76.2167,Hostal Mandala
59,Simao,CN,22.7886,100.9748,Urba Hotel Chacheng Pu'er
125,Arraial do Cabo,BR,-22.9661,-42.0278,Orlanova Boutique Hotel
136,Hambantota,LK,6.1241,81.1185,The Peacock Beach Hotel Hambantota
147,São João da Barra,BR,-21.6403,-41.0511,OYO Pousada Mediterrâneo
201,Cidreira,BR,-30.1811,-50.2056,Hotel Castelo
226,Hong Kong,HK,22.2855,114.1577,The OTTO Hotel
245,General Roca,AR,-39.0333,-67.5833,Estepa Hotel
270,Sittwe,MM,20.1500,92.9000,Hotel Memory
333,Anūpgarh,IN,29.1911,73.2086,Hotel shiva


In [8]:


info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))